## In this notebook:

* on feature set 3

### Imports

In [36]:
#!pip install numpy 
#!pip install pandas
#!pip install torch
#!pip install sklearn

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.optim as optim

### Manipulating data
* Import csv we want
* Map our features to usuable integers
* Map our labels to a one hot encoding

In [37]:
#################################################### 
df = pd.read_pickle("./pickled/chr2-4_featureset3.pkl")
df = df.replace({'-': 0})

# problematic isn't first:

#print("LENGTH",len(df.columns))
col = df.pop("Problematic")
df[col.name] = col

# Split our data into three separate sets for training purposes
train, test = train_test_split(df, shuffle=False)
train, valid = train_test_split(train, shuffle=False)

print(test.head())
print(train.shape, test.shape)

                      aapos  APPRIS TSL codonpos  Ancestral_allele  \
chr pos      ref alt                                                 
3   97791800 T   G      170       5   5        2                 2   
    98580709 G   A      447       5   1        1                 2   
    98581474 T   C      404       5   1        1                 2   
    98585559 G   A      352       5   1        1                 2   
    98585622 G   A      331       5   1        1                 2   

                     HUVEC_fitCons_score      ExAC_cnv.score         GDI  \
chr pos      ref alt                                                       
3   97791800 T   G              0.655142  -0.151628993663861    24.13311   
    98580709 G   A              0.714379   0.240023872494554  3622.54592   
    98581474 T   C              0.714379   0.240023872494554  3622.54592   
    98585559 G   A              0.714379   0.240023872494554  3622.54592   
    98585622 G   A              0.714379   0.24002387

### DataLoader
* Implement the Dataset class supplied by PyTorch
  * Used Map style
  * API: https://pytorch.org/docs/stable/data.html#map-style-datasets
* Instanciate DataLoader using Dataset instance

In [38]:
class DataSetdbNSFP(Dataset):
    def __init__(self, data):
        self.data = torch.FloatTensor(data.values.astype('float'))
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
            target = self.data[index][-1]
            data_val = self.data[index] [:-1]
            return data_val,target
        
train_dataset = DataSetdbNSFP(train)
valid_dataset = DataSetdbNSFP(valid)  
test_dataset = DataSetdbNSFP(test)

train_loader = DataLoader(train_dataset, batch_size=59, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=59, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=59, shuffle=True)

count = 0 
for batch_idx, (data, target) in enumerate(train_loader):
    print('count', count)
    count =+1
    print(data, target)


count 0
tensor([[ 3.3700e+02, -2.0000e+00,  1.0000e+00,  2.0000e+00,  2.0000e+00,
          6.2085e-01,  3.9915e-02,  4.0485e+02,  7.8000e-02,  3.9936e-04,
          1.0000e+00,  2.0000e+00,  1.0000e+01,  1.2000e+01,  2.6000e+01,
          5.8000e+01],
        [ 1.2920e+03,  5.0000e+00,  5.0000e+00,  2.0000e+00,  2.0000e+00,
          5.6410e-01,  0.0000e+00,  4.9002e+03,  8.2500e-02,  7.9872e-04,
          1.0000e+00,  2.0000e+00,  1.1000e+01,  1.4000e+01,  5.7000e+01,
          1.0000e+02],
        [ 2.6400e+02,  0.0000e+00,  1.0000e+00,  1.0000e+00,  2.0000e+00,
          7.2763e-01, -3.9163e-01,  3.7689e+02,  9.7100e-03,  3.9936e-04,
          1.0000e+00,  1.3000e+01,  1.1500e+02,  1.3400e+02,  1.6800e+02,
          3.7200e+02],
        [-1.0000e+00,  0.0000e+00,  5.0000e+00,  0.0000e+00,  2.0000e+00,
          5.7018e-02,  0.0000e+00,  7.4773e+04,  9.7100e-01,  1.7173e-02,
          1.0000e+00,  1.0000e+01,  5.3000e+01,  1.3100e+02,  4.2800e+02,
          7.3400e+02],
        [-1.

tensor([[ 4.7000e+01,  0.0000e+00,  1.0000e+00,  3.0000e+00,  2.0000e+00,
          7.1438e-01,  9.8583e-01,  5.8730e+01,  1.3800e-01,  0.0000e+00,
          1.0000e+00,  1.0000e+00,  1.0000e+00,  4.0000e+00,  1.4000e+01,
          1.4000e+01],
        [ 6.6300e+02,  0.0000e+00,  2.0000e+00,  1.0000e+00,  2.0000e+00,
          6.2085e-01, -1.7286e+00,  2.2568e+03,  2.0700e-02,  7.9872e-04,
          1.0000e+00,  6.2000e+01,  1.6300e+02,  3.7100e+02,  4.3000e+02,
          4.5900e+02],
        [ 1.8970e+03,  0.0000e+00,  1.0000e+00,  1.0000e+00,  2.0000e+00,
          7.1100e-01, -1.0278e+00,  1.7602e+03,  2.2000e-03,  2.3962e-03,
          1.0000e+00,  1.5000e+01,  2.5000e+01,  3.7000e+01,  5.3000e+01,
          9.9000e+01],
        [ 5.2010e+03, -2.0000e+00,  5.0000e+00,  2.0000e+00,  1.0000e+00,
          6.2085e-01,  0.0000e+00,  2.1878e+04,  9.9500e-01,  2.7955e-03,
          1.0000e+00,  3.0000e+00,  2.5000e+01,  4.8000e+01,  1.0000e+02,
          3.1000e+02],
        [ 3.2900e+02

tensor([[ 7.8600e+02,  4.0000e+00,  5.0000e+00,  1.0000e+00,  2.0000e+00,
          7.1438e-01,  3.5203e-01,  5.0046e+02,  1.7400e-02,  0.0000e+00,
          1.0000e+00,  2.0000e+00,  2.0000e+00,  1.3000e+01,  2.5000e+01,
          3.0000e+01],
        [ 4.3600e+02,  0.0000e+00,  2.0000e+00,  2.0000e+00,  2.0000e+00,
          7.1438e-01,  8.7002e-01,  1.0930e+03,  9.6200e-02,  0.0000e+00,
          1.0000e+00,  6.0000e+00,  1.2000e+01,  1.4000e+01,  1.4000e+01,
          1.4000e+01],
        [ 8.0000e+01,  5.0000e+00,  0.0000e+00,  2.0000e+00,  2.0000e+00,
          6.1612e-01, -1.7786e+00,  6.7595e+03,  2.7500e-01,  1.5974e-03,
          1.0000e+00,  2.0000e+00,  5.0000e+00,  1.2000e+01,  3.0000e+01,
          5.9000e+01],
        [ 1.2300e+02,  5.0000e+00,  1.0000e+00,  3.0000e+00,  2.0000e+00,
          7.3541e-01,  4.2924e-01,  1.3428e+01,  3.6400e-02,  0.0000e+00,
          1.0000e+00,  4.5000e+01,  7.6000e+01,  1.3800e+02,  1.3800e+02,
          1.5100e+02],
        [ 1.1522e+04

          3.5200e+02]]) tensor([0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1.,
        1., 0., 0., 1., 1.])
count 1
tensor([[ 6.0000e+01,  0.0000e+00,  5.0000e+00,  2.0000e+00,  2.0000e+00,
          7.1438e-01,  6.5418e-01,  1.2117e+02,  5.1500e-02,  0.0000e+00,
          1.0000e+00,  3.1000e+01,  5.9000e+01,  5.9000e+01,  5.9000e+01,
          1.9600e+02],
        [ 4.2150e+03,  0.0000e+00,  5.0000e+00,  1.0000e+00,  2.0000e+00,
          5.6410e-01,  0.0000e+00,  7.4773e+04,  9.7100e-01,  1.7173e-02,
          1.0000e+00,  1.6000e+01,  3.6000e+01,  7.7000e+01,  1.8400e+02,
          3.4600e+02],
        [ 8.8600e+02,  0.0000e+00,  2.0000e+00,  1.0000e+00,  2.0000e+00,
          7.2763e-01, -3.9163e-01,  3.7689e+02,  9.7100e-03,  3.9936e-04,
          1.0000e+00,  1.1000e+01,  5.5000e+01,  1.1400e+02, 

count 1
tensor([[ 5.9000e+02,  3.0000e+00,  1.0000e+00,  2.0000e+00,  2.0000e+00,
          7.2763e-01, -2.0502e+00,  2.8152e+02,  6.0200e-02,  1.1981e-03,
          1.0000e+00,  4.0000e+00,  1.2000e+01,  1.4000e+01,  2.7000e+01,
          3.3000e+01],
        [-1.0000e+00,  5.0000e+00,  5.0000e+00,  0.0000e+00,  2.0000e+00,
          5.7018e-02,  0.0000e+00,  4.9002e+03,  8.2500e-02,  7.9872e-04,
          1.0000e+00,  9.0000e+00,  1.3000e+01,  1.3000e+01,  2.7000e+01,
          9.2000e+01],
        [ 4.8800e+02, -2.0000e+00,  2.0000e+00,  2.0000e+00,  2.0000e+00,
          7.1100e-01, -1.9885e+00,  2.7758e+03,  3.5900e-01,  4.3930e-03,
          1.0000e+00,  2.0000e+00,  5.0000e+00,  5.0000e+00,  7.0000e+00,
          9.0000e+00],
        [ 4.1500e+02,  0.0000e+00,  1.0000e+00,  1.0000e+00,  2.0000e+00,
          7.1438e-01,  4.5879e-03,  1.0343e+03,  6.4300e-02,  0.0000e+00,
         -1.0000e+00,  2.0000e+00,  3.0000e+00,  9.0000e+00,  1.7000e+01,
          2.5000e+01],
        [ 1.

...

count 1
tensor([[ 1.3300e+02,  0.0000e+00,  3.0000e+00,  3.0000e+00,  2.0000e+00,
          7.5544e-01,  7.1741e-01,  2.6956e+02,  7.0500e-01,  1.5974e-03,
          1.0000e+00,  1.0000e+01,  5.6000e+01,  5.9000e+01,  6.1000e+01,
          8.6000e+01],
        [ 1.4350e+03, -1.0000e+00,  5.0000e+00,  1.0000e+00,  2.0000e+00,
          6.1328e-01,  6.1922e-01,  6.9368e+02,  1.4500e-03,  1.5974e-03,
          1.0000e+00,  3.0000e+00,  3.3000e+01,  4.6000e+01,  7.3000e+01,
          1.5100e+02],
        [ 1.9332e+04,  0.0000e+00,  5.0000e+00,  2.0000e+00,  2.0000e+00,
          5.6410e-01,  0.0000e+00,  7.4773e+04,  9.7100e-01,  1.7173e-02,
          1.0000e+00,  1.4000e+01,  6.7000e+01,  1.1000e+02,  4.3100e+02,
          7.3800e+02],
        [ 1.3320e+03, -2.0000e+00,  0.0000e+00,  2.0000e+00,  2.0000e+00,
          6.1328e-01, -1.7813e-01,  2.9057e+03,  1.2100e-02,  1.1981e-03,
          1.0000e+00,  4.0000e+00,  6.0000e+00,  6.0000e+00,  2.5000e+01,
          7.3000e+01],
        [ 6.

tensor([[ 2.7100e+02,  0.0000e+00,  2.0000e+00,  1.0000e+00,  2.0000e+00,
          7.1438e-01,  2.5558e-01,  1.7152e+03,  1.6500e-03,  3.9936e-04,
          1.0000e+00,  1.0000e+01,  4.1000e+01,  6.4000e+01,  3.3800e+02,
          3.8700e+02],
        [ 1.6500e+02,  0.0000e+00,  3.0000e+00,  1.0000e+00,  2.0000e+00,
          7.1438e-01,  3.2097e-01,  6.4341e+01,  5.9100e-01,  0.0000e+00,
          1.0000e+00,  6.0000e+00,  6.0000e+00,  1.7000e+01,  2.3000e+01,
          2.3000e+01],
        [ 3.1000e+02,  4.0000e+00,  5.0000e+00,  2.0000e+00,  2.0000e+00,
          6.1328e-01, -1.6739e+00,  1.3288e+03,  2.8300e-01,  0.0000e+00,
          1.0000e+00,  2.9000e+01,  2.9000e+01,  3.1000e+01,  3.2000e+01,
          5.7000e+01],
        [ 5.0460e+03, -2.0000e+00,  5.0000e+00,  3.0000e+00, -2.0000e+00,
          6.2085e-01,  0.0000e+00,  2.1878e+04,  9.9500e-01,  2.7955e-03,
          1.0000e+00,  9.0000e+00,  2.9000e+01,  4.7000e+01,  1.1200e+02,
          3.1600e+02],
        [ 6.0900e+02

tensor([[ 1.6741e+04,  0.0000e+00,  5.0000e+00,  2.0000e+00,  2.0000e+00,
          6.2085e-01,  0.0000e+00,  7.4773e+04,  9.7100e-01,  1.7173e-02,
          1.0000e+00,  1.2000e+01,  4.1000e+01,  1.0000e+02,  3.0200e+02,
          7.4700e+02],
        [ 6.8000e+01,  0.0000e+00,  5.0000e+00,  3.0000e+00,  2.0000e+00,
          7.2763e-01, -2.9014e-01,  4.2311e+02,  7.9200e-01,  3.9936e-04,
         -1.0000e+00,  2.0000e+00,  6.0000e+00,  7.0000e+00,  1.9000e+01,
          8.9000e+01],
        [ 8.7800e+02,  3.0000e+00,  1.0000e+00,  2.0000e+00,  2.0000e+00,
          7.1100e-01,  3.5558e-01,  1.8303e+02,  8.0300e-01,  1.5974e-03,
          1.0000e+00,  1.0000e+00,  1.0000e+00,  4.0000e+00,  8.0000e+00,
          4.5000e+01],
        [-1.0000e+00, -2.0000e+00,  1.0000e+00,  0.0000e+00,  2.0000e+00,
          9.2715e-02,  4.0946e-01,  1.1282e+03,  4.1300e-04,  7.9872e-04,
          1.0000e+00,  7.0000e+00,  1.3000e+01,  3.8000e+01,  8.9000e+01,
          1.9200e+02],
        [-1.0000e+00

tensor([[ 2.8060e+03,  0.0000e+00,  5.0000e+00,  1.0000e+00,  2.0000e+00,
          5.4209e-01,  0.0000e+00,  5.0324e+03,  3.7200e-03,  1.5974e-03,
          1.0000e+00,  6.0000e+00,  1.5000e+01,  2.1000e+01,  5.7000e+01,
          1.1700e+02],
        [ 1.0740e+03,  5.0000e+00,  5.0000e+00,  1.0000e+00,  2.0000e+00,
          6.3555e-01, -1.7824e+00,  1.6522e+02,  1.1600e-02,  3.9936e-04,
          1.0000e+00,  9.0000e+00,  1.1000e+01,  1.2000e+01,  3.5000e+01,
          4.6000e+01],
        [ 2.4000e+02, -2.0000e+00,  1.0000e+00,  1.0000e+00,  2.0000e+00,
          5.8640e-01,  3.9915e-02,  4.0485e+02,  7.8000e-02,  3.9936e-04,
          1.0000e+00,  1.0000e+00,  8.0000e+00,  1.6000e+01,  2.8000e+01,
          6.7000e+01],
        [ 1.0700e+02,  0.0000e+00,  2.0000e+00,  1.0000e+00,  2.0000e+00,
          7.1438e-01,  6.5418e-01,  1.2117e+02,  5.1500e-02,  0.0000e+00,
          1.0000e+00,  3.1000e+01,  5.9000e+01,  5.9000e+01,  5.9000e+01,
          1.9600e+02],
        [ 1.8000e+02

count 1
tensor([[ 2.9100e+02, -2.0000e+00,  2.0000e+00,  2.0000e+00,  2.0000e+00,
          6.3526e-01, -5.5058e-01,  2.8273e+02,  1.2600e-01,  0.0000e+00,
          1.0000e+00,  8.6000e+01,  1.0100e+02,  1.1800e+02,  1.1800e+02,
          1.6400e+02],
        [ 5.7000e+01, -2.0000e+00,  1.0000e+00,  1.0000e+00,  2.0000e+00,
          6.3555e-01,  2.3290e-01,  2.2797e+03,  9.0700e-01,  7.9872e-04,
          1.0000e+00,  1.0000e+00,  5.0000e+00,  6.0000e+00,  8.0000e+00,
          1.0000e+01],
        [ 6.1800e+02,  4.0000e+00,  5.0000e+00,  1.0000e+00,  2.0000e+00,
          7.1438e-01,  3.5203e-01,  5.0046e+02,  1.7400e-02,  0.0000e+00,
          1.0000e+00,  8.0000e+00,  1.1000e+01,  1.4000e+01,  2.5000e+01,
          3.0000e+01],
        [ 9.2300e+02,  0.0000e+00,  5.0000e+00,  1.0000e+00,  2.0000e+00,
          5.8640e-01,  0.0000e+00,  7.4773e+04,  9.7100e-01,  1.7173e-02,
          1.0000e+00,  6.0000e+00,  2.4000e+01,  5.7000e+01,  1.1200e+02,
          3.0100e+02],
        [ 4.

### Model
* This model is a simple implementation
* Based on https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [39]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # In: nb of features, out: nb of examples
        self.fc = nn.Linear(16, 1000)
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 1000)
        self.fc4 = nn.Linear(1000, 1000)
         # In: nb of examples, out: nb of predictions
        self.fc5 = nn.Linear(1000, 2)

    def forward(self, x):
        x = x.view((-1, 16))
        h = F.relu(self.fc(x))
        h = self.fc2(h)
        h = F.relu(self.fc2(h))
        h = self.fc3(h)
        h = F.relu(self.fc3(h))
        h = self.fc4(h)
        h = F.relu(self.fc4(h))
        h = self.fc5(h)
        # Softmax to get the actual labels
        return F.softmax(h, dim=1)  ## Note: can potentially use this to get confidence of category predicted 
    
model = Model()
# if cuda:
#     model.cuda()

# Optimizer based on Adam algorithm, uses a slightly lower rate
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Training
* Run a fixed number of epochs
* After each epoch, run the validation set to adjust learning
* Print results
* Based on example: https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [40]:
EPOCHS = 20

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        print(data.shape)
        print(target.shape)
        
#         if cuda:
#             data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        y_pred = model(data) 
        
        # Calculate and appy loss
        loss = F.cross_entropy(y_pred, target.long())
        loss.backward()
        optimizer.step()

        print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1,
            EPOCHS,
            batch_idx * len(data), 
            len(train_loader.dataset),
            100. * batch_idx / len(train_loader), 
            loss.cpu().data.item()))
    
    # NOTE: The Variable class is a wrapper class around a tensor with the added functionalities of back propagation
    # For x, take all cols of datafram except the last one
    evaluate_x = Variable(valid_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
    print(evaluate_x)
    # For y, take last col
    evaluate_y = Variable(valid_loader.dataset.data[:, -1:])
#     if cuda:
#         evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()
    model.eval()
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    
    y_labels = torch.flatten(evaluate_y.data)
    print('Predictions:', pred)
    print('Actual values:', y_labels)
    d = pred.eq(y_labels).cpu()
    accuracy = d.sum().item()*1./d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Validation Accuracy: {:.2f}%'.format(
        epoch+1,
        EPOCHS,
        len(train_loader.dataset), 
        len(train_loader.dataset),
        100. * batch_idx / len(train_loader), 
        loss.cpu().data.item(),
        accuracy*100))

torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [0/10300 (0%)]	Loss: 0.829849
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [59/10300 (1%)]	Loss: 0.596222
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [118/10300 (1%)]	Loss: 0.594315
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [177/10300 (2%)]	Loss: 0.619214
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [236/10300 (2%)]	Loss: 0.569996
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [295/10300 (3%)]	Loss: 0.536924
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [354/10300 (3%)]	Loss: 0.522188
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [413/10300 (4%)]	Loss: 0.538885
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [472/10300 (5%)]	Loss: 0.669454
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [531/10300 (5%)]	Loss: 0.585703
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [590/10300 (6%)]	Loss: 0.550519
torch.Size([59, 16])
tor

 Train Epoch: 1/20 [5369/10300 (52%)]	Loss: 0.516654
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [5428/10300 (53%)]	Loss: 0.567500
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [5487/10300 (53%)]	Loss: 0.518453
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [5546/10300 (54%)]	Loss: 0.720043
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [5605/10300 (54%)]	Loss: 0.533565
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [5664/10300 (55%)]	Loss: 0.618388
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [5723/10300 (55%)]	Loss: 0.635296
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [5782/10300 (56%)]	Loss: 0.584448
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [5841/10300 (57%)]	Loss: 0.703100
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [5900/10300 (57%)]	Loss: 0.533641
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 1/20 [5959/10300 (58%)]	Loss: 0.516652
torch.Size([59, 16])
torch.Size([59])

 Train Epoch: 2/20 [59/10300 (1%)]	Loss: 0.601401
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [118/10300 (1%)]	Loss: 0.635302
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [177/10300 (2%)]	Loss: 0.601401
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [236/10300 (2%)]	Loss: 0.668810
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [295/10300 (3%)]	Loss: 0.533601
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [354/10300 (3%)]	Loss: 0.533600
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [413/10300 (4%)]	Loss: 0.601397
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [472/10300 (5%)]	Loss: 0.669194
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [531/10300 (5%)]	Loss: 0.635296
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [590/10300 (6%)]	Loss: 0.584447
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [649/10300 (6%)]	Loss: 0.753940
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [70

 Train Epoch: 2/20 [5428/10300 (53%)]	Loss: 0.482937
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [5487/10300 (53%)]	Loss: 0.635296
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [5546/10300 (54%)]	Loss: 0.550529
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [5605/10300 (54%)]	Loss: 0.584451
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [5664/10300 (55%)]	Loss: 0.601571
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [5723/10300 (55%)]	Loss: 0.499702
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [5782/10300 (56%)]	Loss: 0.652245
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [5841/10300 (57%)]	Loss: 0.618349
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [5900/10300 (57%)]	Loss: 0.550550
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [5959/10300 (58%)]	Loss: 0.567499
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 2/20 [6018/10300 (58%)]	Loss: 0.550575
torch.Size([59, 16])
torch.Size([59])

 Train Epoch: 3/20 [118/10300 (1%)]	Loss: 0.516652
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 3/20 [177/10300 (2%)]	Loss: 0.652245
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 3/20 [236/10300 (2%)]	Loss: 0.601396
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 3/20 [295/10300 (3%)]	Loss: 0.635263
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 3/20 [354/10300 (3%)]	Loss: 0.601397
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 3/20 [413/10300 (4%)]	Loss: 0.618346
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 3/20 [472/10300 (5%)]	Loss: 0.584448
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 3/20 [531/10300 (5%)]	Loss: 0.550550
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 3/20 [590/10300 (6%)]	Loss: 0.601397
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 3/20 [649/10300 (6%)]	Loss: 0.601397
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 3/20 [708/10300 (7%)]	Loss: 0.584448
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 3/20 [7

...

 Train Epoch: 18/20 [6431/10300 (62%)]	Loss: 0.703092
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 18/20 [6490/10300 (63%)]	Loss: 0.584448
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 18/20 [6549/10300 (63%)]	Loss: 0.533600
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 18/20 [6608/10300 (64%)]	Loss: 0.601397
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 18/20 [6667/10300 (65%)]	Loss: 0.601397
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 18/20 [6726/10300 (65%)]	Loss: 0.601397
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 18/20 [6785/10300 (66%)]	Loss: 0.635296
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 18/20 [6844/10300 (66%)]	Loss: 0.533600
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 18/20 [6903/10300 (67%)]	Loss: 0.584448
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 18/20 [6962/10300 (67%)]	Loss: 0.584448
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 18/20 [7021/10300 (68%)]	Loss: 0.584448
torch.Size([59, 16])
torch

 Train Epoch: 19/20 [1121/10300 (11%)]	Loss: 0.720041
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [1180/10300 (11%)]	Loss: 0.550550
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [1239/10300 (12%)]	Loss: 0.584448
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [1298/10300 (13%)]	Loss: 0.516651
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [1357/10300 (13%)]	Loss: 0.652245
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [1416/10300 (14%)]	Loss: 0.635296
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [1475/10300 (14%)]	Loss: 0.550550
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [1534/10300 (15%)]	Loss: 0.567499
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [1593/10300 (15%)]	Loss: 0.635295
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [1652/10300 (16%)]	Loss: 0.550550
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [1711/10300 (17%)]	Loss: 0.618346
torch.Size([59, 16])
torch

 Train Epoch: 19/20 [6490/10300 (63%)]	Loss: 0.669194
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [6549/10300 (63%)]	Loss: 0.584448
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [6608/10300 (64%)]	Loss: 0.652245
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [6667/10300 (65%)]	Loss: 0.516651
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [6726/10300 (65%)]	Loss: 0.601397
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [6785/10300 (66%)]	Loss: 0.601397
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [6844/10300 (66%)]	Loss: 0.533600
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [6903/10300 (67%)]	Loss: 0.652245
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [6962/10300 (67%)]	Loss: 0.448855
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [7021/10300 (68%)]	Loss: 0.567499
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 19/20 [7080/10300 (69%)]	Loss: 0.533600
torch.Size([59, 16])
torch

 Train Epoch: 20/20 [1121/10300 (11%)]	Loss: 0.482753
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [1180/10300 (11%)]	Loss: 0.601397
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [1239/10300 (12%)]	Loss: 0.635296
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [1298/10300 (13%)]	Loss: 0.465804
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [1357/10300 (13%)]	Loss: 0.465804
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [1416/10300 (14%)]	Loss: 0.584448
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [1475/10300 (14%)]	Loss: 0.601397
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [1534/10300 (15%)]	Loss: 0.567499
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [1593/10300 (15%)]	Loss: 0.550550
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [1652/10300 (16%)]	Loss: 0.686143
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [1711/10300 (17%)]	Loss: 0.550550
torch.Size([59, 16])
torch

 Train Epoch: 20/20 [6431/10300 (62%)]	Loss: 0.584448
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [6490/10300 (63%)]	Loss: 0.635295
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [6549/10300 (63%)]	Loss: 0.703092
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [6608/10300 (64%)]	Loss: 0.635296
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [6667/10300 (65%)]	Loss: 0.499702
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [6726/10300 (65%)]	Loss: 0.567499
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [6785/10300 (66%)]	Loss: 0.533601
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [6844/10300 (66%)]	Loss: 0.635296
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [6903/10300 (67%)]	Loss: 0.550550
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [6962/10300 (67%)]	Loss: 0.618346
torch.Size([59, 16])
torch.Size([59])
 Train Epoch: 20/20 [7021/10300 (68%)]	Loss: 0.652245
torch.Size([59, 16])
torch

### Testing
* Take testing dataset, run it against current model
* Evaluate testing accuracy

In [41]:
# Follow same steps as when we test with validation set, except with the test set
evaluate_x = Variable(test_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
evaluate_y = Variable(test_loader.dataset.data[:, -1:])

model.eval()
output = model(evaluate_x)
pred = output.data.max(1)[1]


y_labels = torch.flatten(evaluate_y.data)
d = pred.eq(y_labels).cpu()
accuracy = d.sum().item()*1./d.size()[0]  ## Accuracy = (TP+TN)/(TP+TN+FP+FN)

print('\n Testing Accuracy: {:.2f}%'.format(

    accuracy*100))

print(len(pred))
print(len(y_labels))


 Testing Accuracy: 68.46%
3434
3434


### Metrics
* Use sklearn builtins to calculate different metrics

In [42]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
print("Confusion matrix: \n", confusion_matrix(y_labels, pred))

# Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_labels, pred))

# Recall
from sklearn.metrics import recall_score
recall = recall_score(y_labels, pred, average=None)
print("Recall TP/(TP+FN):", recall)

# Precision
from sklearn.metrics import precision_score
precision = precision_score(y_labels, pred, average=None)
print("Precision TP/(TP+FP):",precision)


# F1 score
F1 = 2 * (precision * recall) / (precision + recall)
print("F1:", F1)

Confusion matrix: 
 [[   0 1083]
 [   0 2351]]
Accuracy: 0.68462434478742
Recall TP/(TP+FN): [0. 1.]
Precision TP/(TP+FP): [0.         0.68462434]
F1: [      nan 0.8127917]


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


In [43]:
# Recalculating accuracy: TP + TN / (all exmaples)
print("accuracy", 2362/(1080+2362))
1080+2362

#Test shape, train shape
#(10325, 9) (3442, 9)

accuracy 0.6862289366647298


3442